In [1]:
import pandas as pd
import numpy as np
from phik.report import plot_correlation_matrix
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, \
    classification_report
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import f1_score, accuracy_score 
import optuna
from sklearn.metrics import roc_auc_score, average_precision_score
from optuna.pruners import MedianPruner, HyperbandPruner
from tqdm import tqdm
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)

In [2]:
import pandas as pd
import numpy as np
import umap.umap_ as umap
import plotly.graph_objs as go
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier
from sklearn.metrics import roc_auc_score, average_precision_score
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix, roc_auc_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler


In [3]:
def convert_to_snake_case(df):
    df.columns = [col.lower().replace(' ', '_').replace('-', '_') for col in df.columns]
    return df

In [4]:
def onehot_encode_sklearn(df, columns, encoder=None):

    if encoder is None:
        encoder = OneHotEncoder(drop='first', sparse=False)
        encoded_cols = encoder.fit_transform(df[columns])
    else:
        encoded_cols = encoder.transform(df[columns])

    encoded_df = pd.DataFrame(encoded_cols, columns=encoder.get_feature_names_out(columns), index=df.index)

    # Удаляем исходные столбцы и добавляем закодированные
    df = df.drop(columns=columns)
    df = pd.concat([df, encoded_df], axis=1)

    return df, encoder

In [5]:
def scale_data(df, columns=None, scaler=None):

    if columns is None:
        columns = df.columns.tolist()

    if scaler is None:
        scaler = MinMaxScaler()
        scaled_cols = scaler.fit_transform(df[columns])
    else:
        scaled_cols = scaler.transform(df[columns])

    df_scaled = df.copy()
    df_scaled[columns] = scaled_cols

    return df_scaled, scaler

In [6]:
def box_hist_plot(df, class_var, bins=30):
    for i in df.columns.to_list():
        if i != class_var:
            sns.set_style("whitegrid")

            # Создание фигуры с двумя графиками по горизонтали
            fig, ax = plt.subplots(1, 2, figsize=(15, 5))

            # Ящик с усами
            sns.boxplot(data=df, x=i, y=df[class_var], ax=ax[0], orient='h')
            ax[0].set_title(f'Box Plot of {i} by {class_var}')

            # Гистограмма
            sns.histplot(data=df, x=i, hue=df[class_var], ax=ax[1], bins=bins, kde=True)
            ax[1].set_title(f'Histogram of {i} by {class_var}')

            plt.tight_layout()
            plt.show()

In [7]:
# Функция для преобразования РСА
def make_PCA(dataframe, n=2):
    converter_pca = PCA(n_components=n, random_state=42)
    transformed_data = converter_pca.fit_transform(dataframe)

    return transformed_data, converter_pca

In [2]:
df_rides = pd.read_csv(r'../input/competative-data-science-course-by-data-feeling/rides_info.csv')
df_rides

,user_id,car_id,ride_id,ride_date,rating,ride_duration,ride_cost,speed_avg,speed_max,stop_times,distance,refueling,user_ride_quality,deviation_normal
0,o52317055h,A-1049127W,b1v,2020-01-01,4.95,21,268,36,113.548538,0,514.246920,0,1.115260,2.909
1,H41298704y,A-1049127W,T1U,2020-01-01,6.91,8,59,36,93.000000,1,197.520662,0,1.650465,4.133
2,v88009926E,A-1049127W,g1p,2020-01-02,6.01,20,315,61,81.959675,0,1276.328206,0,2.599112,2.461
3,t14229455i,A-1049127W,S1c,2020-01-02,0.26,19,205,32,128.000000,0,535.680831,0,3.216255,0.909
4,W17067612E,A-1049127W,X1b,2020-01-03,1.21,56,554,38,90.000000,1,1729.143367,0,2.716550,-1.822
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
739495,I15732675M,z91796444U,m1z,2020-03-30,7.42,164,2290,49,54.586302,0,2274.772387,0,6.137593,-39.752
739496,r51579957X,z91796444U,t1P,2020-03-30,4.19,26,412,46,73.000000,0,515.580231,0,6.703367,-38.480
739497,l15212727I,z91796444U,z1A,2020-03-31,3.40,23,178,37,71.000000,1,444.846677,0,6.255344,-45.080
739498,Z78949958q,z91796444U,w1R,2020-04-01,9.02,5,75,48,87.000000,1,179.464361,0,7.759949,-60.193


In [3]:
driver_info = pd.read_csv(r'/kaggle/input/competative-data-science-course-by-data-feeling/driver_info.csv')
driver_info#, fix_info.shape

,age,user_rating,user_rides,user_time_accident,user_id,sex,first_ride_date
0,27,9.0,865,19.0,l17437965W,1,2019-4-2
1,46,7.9,2116,11.0,Z12362316j,0,2021-11-19
2,59,7.8,947,4.0,g11098715c,0,2021-1-15
3,37,7.0,18,4.0,U12618125q,0,2019-11-20
4,39,8.2,428,7.0,A14375829B,0,2019-7-23
...,...,...,...,...,...,...,...
15148,19,7.9,52,12.0,R13807707w,1,2019-12-1
15149,40,7.6,1142,3.0,F86199947G,0,2019-1-18
15150,27,7.9,1280,3.0,O10119419p,1,2018-9-16
15151,30,7.9,31,5.0,B35483077Y,0,2019-11-23


In [4]:
driver_rides_merged = df_rides.merge(driver_info, on = 'user_id', how = 'left')

In [5]:
driver_rides_merged

,user_id,car_id,ride_id,ride_date,rating,ride_duration,ride_cost,speed_avg,speed_max,stop_times,distance,refueling,user_ride_quality,deviation_normal,age,user_rating,user_rides,user_time_accident,sex,first_ride_date
0,o52317055h,A-1049127W,b1v,2020-01-01,4.95,21,268,36,113.548538,0,514.246920,0,1.115260,2.909,18,8.0,228,9.0,1,2019-9-16
1,H41298704y,A-1049127W,T1U,2020-01-01,6.91,8,59,36,93.000000,1,197.520662,0,1.650465,4.133,40,7.0,992,9.0,0,2019-2-26
2,v88009926E,A-1049127W,g1p,2020-01-02,6.01,20,315,61,81.959675,0,1276.328206,0,2.599112,2.461,32,7.7,1646,3.0,1,2018-6-11
3,t14229455i,A-1049127W,S1c,2020-01-02,0.26,19,205,32,128.000000,0,535.680831,0,3.216255,0.909,35,7.7,1899,8.0,0,2021-9-22
4,W17067612E,A-1049127W,X1b,2020-01-03,1.21,56,554,38,90.000000,1,1729.143367,0,2.716550,-1.822,24,7.6,472,1.0,1,2019-7-13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
739495,I15732675M,z91796444U,m1z,2020-03-30,7.42,164,2290,49,54.586302,0,2274.772387,0,6.137593,-39.752,39,7.3,817,3.0,0,2019-4-13
739496,r51579957X,z91796444U,t1P,2020-03-30,4.19,26,412,46,73.000000,0,515.580231,0,6.703367,-38.480,33,8.0,357,13.0,1,2020-5-15
739497,l15212727I,z91796444U,z1A,2020-03-31,3.40,23,178,37,71.000000,1,444.846677,0,6.255344,-45.080,18,7.6,1065,8.0,1,2020-11-19
739498,Z78949958q,z91796444U,w1R,2020-04-01,9.02,5,75,48,87.000000,1,179.464361,0,7.759949,-60.193,34,7.9,301,6.0,0,2019-8-26


In [6]:
df_car = pd.read_csv(r'/kaggle/input/competative-data-science-course-by-data-feeling/car_train.csv')

In [7]:
df_car['target_class'].value_counts()

target_class
engine_overheat    289
gear_stick         284
another_bug        279
engine_check       270
break_bug          270
engine_ignition    269
engine_fuel        262
electro_bug        249
wheel_shake        165
Name: count, dtype: int64

In [34]:
# box_hist_plot(df_car[['car_rating', 'year_to_start', 'riders', 'year_to_work',  'target_class']], 'target_class', bins=30)

In [8]:
df_car  

,car_id,model,car_type,fuel_type,car_rating,year_to_start,riders,year_to_work,target_reg,target_class
0,y13744087j,Kia Rio X-line,economy,petrol,3.78,2015,76163,2021,9492.96,another_bug
1,O41613818T,VW Polo VI,economy,petrol,3.90,2015,78218,2021,2656.23,electro_bug
2,d-2109686j,Renault Sandero,standart,petrol,6.30,2012,23340,2017,1526.11,gear_stick
3,u29695600e,Mercedes-Benz GLC,business,petrol,4.04,2011,1263,2020,1338.00,engine_fuel
4,N-8915870N,Renault Sandero,standart,petrol,4.70,2012,26428,2017,825.72,engine_fuel
...,...,...,...,...,...,...,...,...,...,...
2332,j21246192N,Smart ForFour,economy,petrol,4.38,2017,121239,2018,547.30,wheel_shake
2333,h-1554287F,Audi A4,premium,petrol,4.30,2016,107793,2020,26414.19,engine_check
2334,A15262612g,Kia Rio,economy,petrol,3.88,2015,80234,2019,2677.95,gear_stick
2335,W-2514493U,Renault Sandero,standart,petrol,4.50,2014,60048,2020,2213.63,another_bug


In [9]:
f = lambda x: x.nunique()
driver_info_gr = driver_rides_merged.groupby('car_id', as_index=False).agg(
                                                    user_time_accident_max = ('user_time_accident', 'max'),
                                                    user_time_accident_min = ('user_time_accident', 'min'),
                                                    user_time_accident_mean = ('user_time_accident', 'mean'),
                                                age_max = ('age', 'max'),
                                                age_min = ('age', 'min'),
                                                age_mean = ('age', 'mean'),
                                                    sex_sum = ('sex', 'sum'),
                                                user_rating_max = ('user_rides', 'max'),
                                                user_rating_min = ('user_rides', 'min'),
                                                user_rating_mean = ('user_rides', 'mean'),
                                                    user_rides_max = ('user_rides', 'max'),
                                                    user_rides_min = ('user_rides', 'min'),
                                                    user_rides_mean = ('user_rides', 'mean'),
                                                distance_sum = ('distance', 'sum'),
                                                   mean_rating = ('rating', 'mean'),
                                                   rating_min = ('rating', 'min'),
                                                   rating_max = ('rating', 'max'),
                                               speed_max = ('speed_max', 'max'),
                                               speed_avg = ('speed_avg', 'max'),
                                                   ride_duration_max = ('ride_duration', 'max'),
                                                   ride_duration_mean = ('ride_duration', 'mean'),
                                                   ride_duration_sum = ('ride_duration', 'sum'),
                                               user_ride_quality_median = ('user_ride_quality', 'median'), 
                                               user_ride_quality_min = ('user_ride_quality', 'min'),
                                               user_ride_quality_max = ('user_ride_quality', 'max'),
                                                   deviation_normal_min = ('deviation_normal', 'min'),
                                                   deviation_normal_max = ('deviation_normal', 'max'),
                                                   deviation_normal_mean = ('deviation_normal', 'mean'),
                                               user_uniq = ('user_id', f))

In [10]:
car_driver_rides_merged = df_car.merge(driver_info_gr, on = 'car_id', how = 'left')

In [11]:
car_driver_rides_merged

,car_id,model,car_type,fuel_type,car_rating,year_to_start,riders,year_to_work,target_reg,target_class,...,ride_duration_max,ride_duration_mean,ride_duration_sum,user_ride_quality_median,user_ride_quality_min,user_ride_quality_max,deviation_normal_min,deviation_normal_max,deviation_normal_mean,user_uniq
0,y13744087j,Kia Rio X-line,economy,petrol,3.78,2015,76163,2021,9492.96,another_bug,...,37392,1635.770115,284624,0.023174,-10.501738,11.035871,-9.000,0.001,-0.120391,170
1,O41613818T,VW Polo VI,economy,petrol,3.90,2015,78218,2021,2656.23,electro_bug,...,38450,2223.178161,386833,12.306011,0.437053,32.610351,-25.088,47.673,6.050011,174
2,d-2109686j,Renault Sandero,standart,petrol,6.30,2012,23340,2017,1526.11,gear_stick,...,39922,2048.856322,356501,2.513319,-12.535368,11.766087,-12.400,4.001,-2.223954,173
3,u29695600e,Mercedes-Benz GLC,business,petrol,4.04,2011,1263,2020,1338.00,engine_fuel,...,39725,1943.511494,338171,-5.029476,-10.723544,3.931040,-12.691,48.956,14.771948,170
4,N-8915870N,Renault Sandero,standart,petrol,4.70,2012,26428,2017,825.72,engine_fuel,...,36872,1597.793103,278016,-14.260456,-25.742137,3.344463,-20.907,49.269,12.455678,171
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2332,j21246192N,Smart ForFour,economy,petrol,4.38,2017,121239,2018,547.30,wheel_shake,...,39606,2317.402299,403228,-6.624534,-21.867121,16.144112,-10.842,20.808,2.914989,171
2333,h-1554287F,Audi A4,premium,petrol,4.30,2016,107793,2020,26414.19,engine_check,...,35278,1331.701149,231716,-8.582467,-17.949058,3.118135,-49.851,5.330,-18.805856,169
2334,A15262612g,Kia Rio,economy,petrol,3.88,2015,80234,2019,2677.95,gear_stick,...,33901,1674.942529,291440,-0.778524,-12.441852,13.806006,-65.503,41.290,-7.651776,172
2335,W-2514493U,Renault Sandero,standart,petrol,4.50,2014,60048,2020,2213.63,another_bug,...,37219,2384.229885,414856,2.464975,-12.514535,12.061994,-25.038,26.106,0.468718,171


In [12]:
fix_info = pd.read_csv(r'/kaggle/input/competative-data-science-course-by-data-feeling/fix_info.csv')
fix_info.sample(5)#, fix_info.shape

,car_id,worker_id,fix_date,work_type,destroy_degree,work_duration
173,d-6672088A,HJ,2020-4-5 15:4,reparking,1.0,33
139785,L-6391483w,RT,2019-12-30 16:43,reparking,1.0,31
41745,j22675199u,NL,2019-3-17 5:56,reparking,1.0,24
72178,z41744928r,ML,2019-5-2 4:14,repair,3.0,3
116180,j55128828K,AO,2020-10-4 7:15,repair,5.2,17


In [13]:
f = lambda x: x.nunique()
fix_info_gr = fix_info.groupby('car_id', as_index=False).agg(
#                                                    work_type = ('work_type', 'max'),
                                               destroy_degree_max = ('destroy_degree', 'max'),
                                               destroy_degree_min = ('destroy_degree', 'min'),
                                               destroy_degree_mean = ('destroy_degree', 'mean'),
                                                   work_duration_max = ('work_duration', 'max'),
                                                   work_duration_min = ('work_duration', 'min'),
                                                   work_duration_mean = ('work_duration', 'mean'),
                                               worker_uniq = ('worker_id', f))

In [14]:
fix_info_gr

,car_id,destroy_degree_max,destroy_degree_min,destroy_degree_mean,work_duration_max,work_duration_min,work_duration_mean,worker_uniq
0,A-1049127W,9.0,1.0,2.714286,56,1,27.485714,35
1,A-1079539w,10.0,1.0,4.329412,64,3,27.382353,33
2,A-1162143G,10.0,1.0,3.705882,69,1,27.823529,34
3,A-1228282M,5.0,1.0,2.908824,65,3,24.647059,33
4,A-1339912r,5.0,1.0,3.044118,71,3,28.617647,33
...,...,...,...,...,...,...,...,...
4245,z73688663a,5.0,1.0,2.705714,59,11,26.028571,33
4246,z73740510r,8.0,1.0,3.714286,49,3,25.457143,35
4247,z86911952C,10.0,1.0,3.417143,48,1,26.942857,34
4248,z91353693Z,10.0,1.0,4.497059,68,1,28.764706,32


In [15]:
merged_data = car_driver_rides_merged.merge(fix_info_gr, on = 'car_id', how = 'left')

In [16]:
# merged_data = df_car.merge(df_rides, on = 'car_id', how = 'left')
# merged_data = df_car.copy()
merged_data

,car_id,model,car_type,fuel_type,car_rating,year_to_start,riders,year_to_work,target_reg,target_class,...,deviation_normal_max,deviation_normal_mean,user_uniq,destroy_degree_max,destroy_degree_min,destroy_degree_mean,work_duration_max,work_duration_min,work_duration_mean,worker_uniq
0,y13744087j,Kia Rio X-line,economy,petrol,3.78,2015,76163,2021,9492.96,another_bug,...,0.001,-0.120391,170,9.0,1.0,3.048571,56,7,26.657143,33
1,O41613818T,VW Polo VI,economy,petrol,3.90,2015,78218,2021,2656.23,electro_bug,...,47.673,6.050011,174,10.0,1.0,2.917143,48,4,24.942857,34
2,d-2109686j,Renault Sandero,standart,petrol,6.30,2012,23340,2017,1526.11,gear_stick,...,4.001,-2.223954,173,10.0,1.0,3.740000,59,1,26.142857,35
3,u29695600e,Mercedes-Benz GLC,business,petrol,4.04,2011,1263,2020,1338.00,engine_fuel,...,48.956,14.771948,170,10.0,1.0,4.085714,64,1,28.771429,34
4,N-8915870N,Renault Sandero,standart,petrol,4.70,2012,26428,2017,825.72,engine_fuel,...,49.269,12.455678,171,10.0,1.0,3.880000,65,10,28.028571,34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2332,j21246192N,Smart ForFour,economy,petrol,4.38,2017,121239,2018,547.30,wheel_shake,...,20.808,2.914989,171,10.0,1.0,3.900000,71,4,23.941176,32
2333,h-1554287F,Audi A4,premium,petrol,4.30,2016,107793,2020,26414.19,engine_check,...,5.330,-18.805856,169,5.0,1.0,1.935294,63,2,25.676471,34
2334,A15262612g,Kia Rio,economy,petrol,3.88,2015,80234,2019,2677.95,gear_stick,...,41.290,-7.651776,172,7.4,1.0,2.782353,44,6,23.441176,31
2335,W-2514493U,Renault Sandero,standart,petrol,4.50,2014,60048,2020,2213.63,another_bug,...,26.106,0.468718,171,10.0,1.0,3.441176,64,1,25.205882,34


In [17]:
merged_data['car_age'] = 2022 - merged_data['year_to_start']
merged_data['car_years_at_work'] = 2022 - merged_data['year_to_work']
merged_data['car_age_to_work'] = merged_data['car_age'] / merged_data['car_years_at_work']

In [46]:
merged_data['car_years_at_work'].max(), merged_data['car_age'].max(),

(8, 11)

In [18]:
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2337 entries, 0 to 2336
Data columns (total 49 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   car_id                    2337 non-null   object 
 1   model                     2337 non-null   object 
 2   car_type                  2337 non-null   object 
 3   fuel_type                 2337 non-null   object 
 4   car_rating                2337 non-null   float64
 5   year_to_start             2337 non-null   int64  
 6   riders                    2337 non-null   int64  
 7   year_to_work              2337 non-null   int64  
 8   target_reg                2337 non-null   float64
 9   target_class              2337 non-null   object 
 10  user_time_accident_max    2337 non-null   float64
 11  user_time_accident_min    2337 non-null   float64
 12  user_time_accident_mean   2337 non-null   float64
 13  age_max                   2337 non-null   int64  
 14  age_min 

## Start of  Первый этап!

In [19]:
first_3_important_feature = ['user_time_accident_mean',
              'mean_rating',
                  'speed_max',
        'destroy_degree_max',
    'user_time_accident_max',
                'rating_min',
                  'speed_avg',
       'work_duration_max',
       'destroy_degree_mean',
                   'model']

In [20]:
from sklearn.model_selection import train_test_split

X = merged_data[first_3_important_feature]#.drop(['target_class', 'target_reg'], axis=1).copy() #
y = merged_data['target_class'].copy() #


# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
X_train.shape, X_test.shape

((1869, 10), (468, 10))

In [21]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42, stratify=y_train)
X_train.shape, X_val.shape, y_train.shape, y_val.shape

((1495, 10), (374, 10), (1495,), (374,))

In [22]:
baseline_cat_features = ['car_id', 'car_type', 'fuel_type', 'model',
#                 'user_id',
#                 'ride_id', 
#                 'ride_date'
               ]

In [88]:
baseline_cat_features = ['model']

In [89]:
import optuna
from sklearn.metrics import roc_auc_score, average_precision_score


def baseline_objective(trial):
    params = {
        'iterations': trial.suggest_int('iterations', 1000, 4000),
        'boosting_type': trial.suggest_categorical('boosting_type', [  'Plain' ]), # 'Ordered',
        'depth': trial.suggest_int('depth', 3, 11),
        'learning_rate': trial.suggest_float('learning_rate', 0.05, 0.1 ), #  0.05, 0.1
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 2, 15),
        'border_count': trial.suggest_int('border_count', 20, 300),
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0, 9),
        'od_type': 'Iter', 
        'od_wait': 100, 
        'eval_metric': 'MultiClass',
        'logging_level': 'Silent',
        'random_seed': 42,
        'loss_function': 'MultiClass'
#         'task_type': 'GPU'
    }

    baseline_model = CatBoostClassifier(**params)
    baseline_model.fit(X_train, y_train, 
                               eval_set=[(X_val, y_val)],
                               use_best_model=True, 
                               early_stopping_rounds=params['od_wait'], 
                               cat_features=baseline_cat_features)

    baseline_y_pred = baseline_model.predict(X_test)
        
    baseline_accuracy = accuracy_score(y_test, baseline_y_pred)
#     avg_prec = average_precision_score(y_test, y_pred)
    
    return baseline_accuracy

In [90]:
%%time 
baseline_study = optuna.create_study(pruner=HyperbandPruner(),
                            directions=['maximize'],
                           sampler=optuna.samplers.TPESampler(seed=42)) #pruner=MedianPruner(), sampler=optuna.samplers.TPESampler(seed=42)
baseline_study.optimize(baseline_objective,
               n_jobs=-1,
               n_trials=10, # 25 показывает результат лучше
               show_progress_bar=True
              )

baseline_best_trial = baseline_study.best_trials[0]
baseline_accuracy_best = baseline_best_trial.values[0]
# avg_prec_best = best_trial.values[1]

print(f"Лучшее значение  accuracy: {baseline_accuracy_best:.4f}")
# print(f"Лучшее значение Average Precision: {avg_prec_best:.4f}")

[I 2023-11-14 14:39:22,713] A new study created in memory with name: no-name-8ecdff4a-4734-428e-ba6e-24c8d1833049


  0%|          | 0/10 [00:00<?, ?it/s]

[I 2023-11-14 14:40:37,415] Trial 2 finished with value: 0.9850427350427351 and parameters: {'iterations': 1889, 'boosting_type': 'Plain', 'depth': 5, 'learning_rate': 0.09290695239175306, 'l2_leaf_reg': 4.648123480248932, 'border_count': 135, 'bagging_temperature': 4.6671020453566054}. Best is trial 2 with value: 0.9850427350427351.
[I 2023-11-14 14:41:14,538] Trial 3 finished with value: 0.9829059829059829 and parameters: {'iterations': 3715, 'boosting_type': 'Plain', 'depth': 4, 'learning_rate': 0.0877942194556745, 'l2_leaf_reg': 10.14003013305619, 'border_count': 141, 'bagging_temperature': 6.376510281264394}. Best is trial 2 with value: 0.9850427350427351.
[I 2023-11-14 14:41:46,573] Trial 1 finished with value: 0.9850427350427351 and parameters: {'iterations': 3600, 'boosting_type': 'Plain', 'depth': 5, 'learning_rate': 0.08008486253429836, 'l2_leaf_reg': 9.259096851747238, 'border_count': 70, 'bagging_temperature': 5.960638418185166}. Best is trial 2 with value: 0.98504273504273

In [91]:
%%time
# Возьмите параметры из лучшего trial на основе первой метрики (ROC AUC в данном случае)
baseline_best_params = baseline_study.best_trials[0].params

baseline_CatBoost_model = CatBoostClassifier(**baseline_best_params,
                                    eval_metric='MultiClass',
#                                     auto_class_weights='Balanced',
                                    logging_level='Silent',
                                    use_best_model=True) #  task_type='GPU' auto_class_weights='Balanced' , 

baseline_CatBoost_model.fit(X_train, y_train,
                           eval_set=[(X_val, y_val)],
                           early_stopping_rounds=100,
                           verbose=100, cat_features=baseline_cat_features)

baseline_y_pred = baseline_CatBoost_model.predict(X_test)
baseline_best_accuracy = accuracy_score(y_test, baseline_y_pred)
print(f"Лучшее значение  accuracy: {baseline_best_accuracy:.4f}")

Лучшее значение  accuracy: 0.9850
CPU times: user 2min 1s, sys: 8.99 s, total: 2min 10s
Wall time: 37.9 s


In [95]:
# baseline_CatBoost_model.save_model("catboost_best_model_01_824.cbm")
# import json

# with open("catboost_best_parameters_01_824.json", "w") as f:
#     json.dump(baseline_best_params, f)

In [56]:
fi = baseline_CatBoost_model.get_feature_importance(prettified=True)
fi

,Feature Id,Importances
0,user_time_accident_mean,19.314387
1,mean_rating,18.947303
2,speed_max,18.412096
3,destroy_degree_max,14.209886
4,user_time_accident_max,10.581023
5,rating_min,3.018036
6,speed_avg,1.411158
7,work_duration_max,1.404726
8,destroy_degree_mean,1.316987
9,model,1.013498


In [79]:
first_3_important_feature = fi['Feature Id'][:10]
# first_3_important_feature = ['speed_max', 'mean_rating', 'rating_min']
first_3_important_feature

0    user_time_accident_mean
1                mean_rating
2                  speed_max
3         destroy_degree_max
4     user_time_accident_max
5                 rating_min
6                  speed_avg
7          work_duration_max
8        destroy_degree_mean
9                      model
Name: Feature Id, dtype: object

In [76]:
first_3_important_feature

0    user_time_accident_mean
1                mean_rating
2                  speed_max
3         destroy_degree_max
4     user_time_accident_max
5                 rating_min
6                  speed_avg
7          work_duration_max
8        destroy_degree_mean
9                      model
Name: Feature Id, dtype: object

## Second stage Start!

In [23]:
# iter_best_accuracy = baseline_best_accuracy.copy()
iter_best_accuracy = 0.985
iter_best_accuracy

0.985

In [24]:
# new_features = fi['Feature Id'][10:].to_list()

new_features = ['user_ride_quality_max',
 'sex_sum',
 'rating_max',
 'car_type',
 'deviation_normal_max',
 'deviation_normal_min',
 'deviation_normal_mean',
 'age_mean',
 'riders',
 'user_ride_quality_min',
 'user_uniq',
 'car_rating',
 'work_duration_mean',
 'ride_duration_max',
 'car_age_to_work',
 'distance_sum',
 'age_max',
 'user_rating_mean',
 'user_rating_max',
 'ride_duration_mean',
 'user_ride_quality_median',
 'user_rides_min',
 'work_duration_min',
 'user_rating_min',
 'user_rides_max',
 'user_rides_mean',
 'worker_uniq',
 'car_years_at_work',
 'ride_duration_sum',
 'year_to_work',
 'year_to_start',
 'age_min',
 'user_time_accident_min',
 'car_age',
 'fuel_type',
 'car_id',
 'destroy_degree_min']
new_features

['user_ride_quality_max',
 'sex_sum',
 'rating_max',
 'car_type',
 'deviation_normal_max',
 'deviation_normal_min',
 'deviation_normal_mean',
 'age_mean',
 'riders',
 'user_ride_quality_min',
 'user_uniq',
 'car_rating',
 'work_duration_mean',
 'ride_duration_max',
 'car_age_to_work',
 'distance_sum',
 'age_max',
 'user_rating_mean',
 'user_rating_max',
 'ride_duration_mean',
 'user_ride_quality_median',
 'user_rides_min',
 'work_duration_min',
 'user_rating_min',
 'user_rides_max',
 'user_rides_mean',
 'worker_uniq',
 'car_years_at_work',
 'ride_duration_sum',
 'year_to_work',
 'year_to_start',
 'age_min',
 'user_time_accident_min',
 'car_age',
 'fuel_type',
 'car_id',
 'destroy_degree_min']

In [25]:
X_train_kopiya = X_train[first_3_important_feature].copy()
X_val_kopiya = X_val[first_3_important_feature].copy()
X_test_kopiya = X_test[first_3_important_feature].copy()

X_train_kopiya.shape, X_val_kopiya.shape, X_test_kopiya.shape

((1495, 10), (374, 10), (468, 10))

In [26]:
cat_features_old = ['car_id', 'car_type', 'fuel_type', 'model',
#                 'user_id',
#                 'ride_id', 
#                 'ride_date'
               ]

In [159]:
cat_features_att = [ i for i in cat_features_old if i in X_train_kopiya.columns.to_list() or i in [feature] ]
cat_features_att

['car_id']

In [96]:
# feature =  'car_years_at_work' # 'worker_uniq' #

# X_train_temp = X_train_kopiya.copy()
# X_val_temp = X_val_kopiya.copy()
# X_test_temp = X_test_kopiya.copy()
        
# X_train_temp[feature] = merged_data.loc[X_train.index, feature].copy()
# X_val_temp[feature] = merged_data.loc[X_val.index, feature].copy()
# X_test_temp[feature] = merged_data.loc[X_test.index, feature].copy()

# baseline_CatBoost_model.fit(X_train_temp, y_train,
#                                eval_set=[(X_val_temp, y_val)], 
#                                early_stopping_rounds=100,
#                                verbose=100,
#                                cat_features=cat_features_temp)

# # Оценка производительности
# current_y_pred = baseline_CatBoost_model.predict(X_test_temp)
# accuracy_score_current = accuracy_score(y_test, current_y_pred)
# print(f'Feature tested: {feature}, accuracy_score_current: {accuracy_score_current}\n')


In [ ]:
from sklearn.metrics import accuracy_score 
from catboost import CatBoostClassifier


initial_best_f1 = iter_best_accuracy  # F1-мера исходной модели
threshold_f1 = 1.0  # Заданный порог для F1-меры
best_features = []

while new_features:
    while_iter_best_f1 = initial_best_f1  # Инициализация лучшей F1-меры текущим лучшим значением
    feature_performance_dict = {}  # Словарь для хранения производительности признаков
    
    print(f'<<<<<< while_iter_best_F1_Score = {while_iter_best_f1} >>>>>>\n')
    
    for feature in tqdm(new_features):
        print(f'Feature Added: {feature} \n')
        
        # Копирование исходных данных
        X_train_temp = X_train_kopiya.copy()
        X_val_temp = X_val_kopiya.copy()
        X_test_temp = X_test_kopiya.copy()

        # Добавление текущего признака
        X_train_temp[feature] = merged_data.loc[X_train.index, feature]
        X_val_temp[feature] = merged_data.loc[X_val.index, feature]
        X_test_temp[feature] = merged_data.loc[X_test.index, feature]
        
        cat_features_temp = [ i for i in cat_features_old if i in X_train_temp.columns.to_list() ]
        print(f'Current Cat_features_temp -> {cat_features_temp}')
                
        # Обучение модели
        CatBoost_model_temp = CatBoostClassifier(n_estimators = 1000,
                                                 eval_metric='MultiClass',
                                                 loss_function='MultiClass',
                                                 random_seed=42)
        CatBoost_model_temp.fit(X_train_temp, y_train,
                                       eval_set=[(X_val_temp, y_val)], 
                                       early_stopping_rounds=100,
                                       verbose=100,
                                       cat_features=cat_features_temp)

        # Оценка производительности
        current_y_pred = CatBoost_model_temp.predict(X_test_temp)
        accuracy_score_current = accuracy_score(y_test, current_y_pred)
        
        print(f'Последние 5 колонок в датасете X_train_temp: {X_train_temp.columns.to_list()[-5:]} Len X_train_temp is {X_train_temp.shape[1]}\n')
        print(f'Feature tested: {feature}, accuracy_score_current: {accuracy_score_current}\n')
        
        # Запись результатов в словарь
        feature_performance_dict[feature] = accuracy_score_current

    # Конвертация словаря в DataFrame и сортировка по F1-Score
    feature_performance = pd.DataFrame(list(feature_performance_dict.items()), columns=['Feature', 'F1_Score'])
    feature_performance.sort_values(by='F1_Score', ascending=False, inplace=True)
    
    print(feature_performance.sort_values(by='F1_Score', ascending=False), '\n')
    
    # Проверка наличия улучшения и добавление лучшего признака
    if not feature_performance.empty and (feature_performance.iloc[0]['F1_Score'] > while_iter_best_f1 or feature_performance.iloc[0]['F1_Score'] >= threshold_f1):
        best_feature = feature_performance.iloc[0]['Feature']
        
        # Добавление лучшего признака к исходным данным
        X_train_kopiya[best_feature] = merged_data.loc[X_train.index, best_feature]
        X_val_kopiya[best_feature] = merged_data.loc[X_val.index, best_feature]
        X_test_kopiya[best_feature] = merged_data.loc[X_test.index, best_feature]

        # Обновление списка лучших признаков
        best_features.append(best_feature)
        initial_best_f1 = feature_performance.iloc[0]['F1_Score']
        
        # Удаление добавленного признака из списка новых признаков
        new_features.remove(best_feature)

    else:
        # Если не найдено улучшений, прерываем цикл
        break

print(f'Процесс проверки признаков {new_features[:20]} закончен!')

<<<<<< while_iter_best_F1_Score = 0.985 >>>>>>



  0%|          | 0/37 [00:00<?, ?it/s]

Feature Added: user_ride_quality_max 

Current Cat_features_temp -> ['model']
Learning rate set to 0.108848
0:	learn: 1.5902126	test: 1.5774009	best: 1.5774009 (0)	total: 94.9ms	remaining: 1m 34s
100:	learn: 0.0412344	test: 0.0276131	best: 0.0276131 (100)	total: 2.48s	remaining: 22.1s
200:	learn: 0.0223951	test: 0.0169102	best: 0.0169102 (200)	total: 4.9s	remaining: 19.5s
300:	learn: 0.0140970	test: 0.0127127	best: 0.0127127 (300)	total: 7.27s	remaining: 16.9s
400:	learn: 0.0099185	test: 0.0105400	best: 0.0105400 (400)	total: 9.63s	remaining: 14.4s
500:	learn: 0.0074095	test: 0.0088881	best: 0.0088577 (499)	total: 12s	remaining: 12s
600:	learn: 0.0059420	test: 0.0081728	best: 0.0081728 (600)	total: 14.4s	remaining: 9.58s
700:	learn: 0.0048793	test: 0.0074452	best: 0.0074346 (696)	total: 16.8s	remaining: 7.17s
800:	learn: 0.0041447	test: 0.0070732	best: 0.0070520 (798)	total: 19.2s	remaining: 4.76s
900:	learn: 0.0036132	test: 0.0067847	best: 0.0067804 (899)	total: 21.6s	remaining: 2.37s

  3%|▎         | 1/37 [00:24<14:45, 24.61s/it]

999:	learn: 0.0031936	test: 0.0065704	best: 0.0065617 (971)	total: 24.3s	remaining: 0us

bestTest = 0.006561698453
bestIteration = 971

Shrink model to first 972 iterations.
Последние 5 колонок в датасете X_train_temp: ['speed_avg', 'work_duration_max', 'destroy_degree_mean', 'model', 'user_ride_quality_max'] Len X_train_temp is 11

Feature tested: user_ride_quality_max, accuracy_score_current: 0.9850427350427351

Feature Added: sex_sum 

Current Cat_features_temp -> ['model']
Learning rate set to 0.108848
0:	learn: 1.5902126	test: 1.5774009	best: 1.5774009 (0)	total: 28.9ms	remaining: 28.9s
100:	learn: 0.0392465	test: 0.0257039	best: 0.0257039 (100)	total: 2.35s	remaining: 20.9s
200:	learn: 0.0211490	test: 0.0158121	best: 0.0157987 (199)	total: 4.63s	remaining: 18.4s
300:	learn: 0.0139593	test: 0.0120300	best: 0.0120300 (300)	total: 6.91s	remaining: 16s
400:	learn: 0.0097290	test: 0.0092733	best: 0.0092733 (400)	total: 9.19s	remaining: 13.7s
500:	learn: 0.0073873	test: 0.0081103	best:

...

999:	learn: 0.0032565	test: 0.0083061	best: 0.0082008 (928)	total: 23.3s	remaining: 0us

bestTest = 0.008200828756
bestIteration = 928

Shrink model to first 929 iterations.
Последние 5 колонок в датасете X_train_temp: ['speed_avg', 'work_duration_max', 'destroy_degree_mean', 'model', 'age_max'] Len X_train_temp is 11

Feature tested: age_max, accuracy_score_current: 0.9871794871794872

Feature Added: user_rating_mean 

Current Cat_features_temp -> ['model']
Learning rate set to 0.108848
0:	learn: 1.5902126	test: 1.5774009	best: 1.5774009 (0)	total: 27.6ms	remaining: 27.6s
100:	learn: 0.0429661	test: 0.0290511	best: 0.0290511 (100)	total: 2.48s	remaining: 22s
200:	learn: 0.0221683	test: 0.0172208	best: 0.0172208 (200)	total: 4.88s	remaining: 19.4s
300:	learn: 0.0141122	test: 0.0133153	best: 0.0133057 (299)	total: 7.28s	remaining: 16.9s
400:	learn: 0.0103041	test: 0.0116729	best: 0.0116729 (400)	total: 10s	remaining: 14.9s
500:	learn: 0.0078237	test: 0.0100901	best: 0.0100901 (500)	tota

 49%|████▊     | 18/37 [07:36<07:40, 24.24s/it]

999:	learn: 0.0034168	test: 0.0072587	best: 0.0072503 (996)	total: 24.4s	remaining: 0us

bestTest = 0.007250311248
bestIteration = 996

Shrink model to first 997 iterations.
Последние 5 колонок в датасете X_train_temp: ['speed_avg', 'work_duration_max', 'destroy_degree_mean', 'model', 'user_rating_mean'] Len X_train_temp is 11

Feature tested: user_rating_mean, accuracy_score_current: 0.9829059829059829

Feature Added: user_rating_max 

Current Cat_features_temp -> ['model']
Learning rate set to 0.108848
0:	learn: 1.5902126	test: 1.5774009	best: 1.5774009 (0)	total: 25.7ms	remaining: 25.6s
100:	learn: 0.0400910	test: 0.0262099	best: 0.0262099 (100)	total: 2.44s	remaining: 21.7s
200:	learn: 0.0224765	test: 0.0176369	best: 0.0176369 (200)	total: 4.82s	remaining: 19.2s
300:	learn: 0.0143769	test: 0.0138212	best: 0.0137902 (297)	total: 7.21s	remaining: 16.8s
400:	learn: 0.0103397	test: 0.0119243	best: 0.0119022 (394)	total: 9.6s	remaining: 14.3s
500:	learn: 0.0079560	test: 0.0105343	best: 

 51%|█████▏    | 19/37 [08:00<07:16, 24.27s/it]

999:	learn: 0.0034501	test: 0.0077510	best: 0.0077345 (996)	total: 24.1s	remaining: 0us

bestTest = 0.007734516619
bestIteration = 996

Shrink model to first 997 iterations.
Последние 5 колонок в датасете X_train_temp: ['speed_avg', 'work_duration_max', 'destroy_degree_mean', 'model', 'user_rating_max'] Len X_train_temp is 11

Feature tested: user_rating_max, accuracy_score_current: 0.9850427350427351

Feature Added: ride_duration_mean 

Current Cat_features_temp -> ['model']
Learning rate set to 0.108848
0:	learn: 1.5902126	test: 1.5774009	best: 1.5774009 (0)	total: 25.8ms	remaining: 25.8s
100:	learn: 0.0419494	test: 0.0288490	best: 0.0288490 (100)	total: 2.5s	remaining: 22.3s
200:	learn: 0.0219048	test: 0.0177665	best: 0.0177286 (199)	total: 5s	remaining: 19.9s
300:	learn: 0.0136631	test: 0.0139023	best: 0.0138931 (298)	total: 7.38s	remaining: 17.1s
400:	learn: 0.0101323	test: 0.0119823	best: 0.0119564 (399)	total: 9.76s	remaining: 14.6s
500:	learn: 0.0078291	test: 0.0109549	best: 0.

In [64]:
X_test_kopiya.columns.to_list()

['mean_rating',
 'speed_max',
 'destroy_degree_max',
 'rating_min',
 'model',
 'car_type',
 'speed_avg',
 'rating_max',
 'fuel_type',
 'deviation_normal_max',
 'destroy_degree_min',
 'riders',
 'user_uniq']

## END of  Второй этап!

In [65]:
chusen_features = X_test_kopiya.columns.to_list()[:-2]
chusen_features

['mean_rating',
 'speed_max',
 'destroy_degree_max',
 'rating_min',
 'model',
 'car_type',
 'speed_avg',
 'rating_max',
 'fuel_type',
 'deviation_normal_max',
 'destroy_degree_min']

In [ ]:
['mean_rating',
 'speed_max',
 'destroy_degree_max',
 'rating_min',
 'model',
 'car_type',
 'speed_avg',
 'rating_max',
 'fuel_type',
 'deviation_normal_max',
 'destroy_degree_min']

In [66]:
from sklearn.model_selection import train_test_split

X = merged_data[chusen_features].copy()#.drop(['smoking'], axis=1).copy() #
y = merged_data['target_class'].copy() #

# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42, stratify=y)
X_train.shape, X_test.shape

((1986, 11), (351, 11))

In [67]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42, stratify=y_train)

X_train.shape, X_val.shape, y_train.shape, y_val.shape

((1588, 11), (398, 11), (1588,), (398,))

In [ ]:
['fuel_type', 'model', 'car_type']

In [71]:
import optuna
from sklearn.metrics import roc_auc_score, average_precision_score


def chusen_objective(trial):
    params = {
        'iterations': trial.suggest_int('iterations', 1000, 3000),
        'boosting_type': trial.suggest_categorical('boosting_type', [ 'Plain' ]), # 'Ordered',
        'depth': trial.suggest_int('depth', 3, 11),
        'learning_rate': trial.suggest_float('learning_rate', 0.05, 0.1 ), #  0.05, 0.1
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 2, 15),
        'border_count': trial.suggest_int('border_count', 20, 300),
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0, 9),
        'od_type': 'Iter', 
        'od_wait': 200, 
        'eval_metric': 'MultiClass',
        'logging_level': 'Silent',
        'random_seed': 42,
        'loss_function': 'MultiClass'
#         'task_type': 'GPU'
    }

    chusen_model = CatBoostClassifier(**params)
    chusen_model.fit(X_train, y_train, eval_set=[(X_val, y_val)], use_best_model=True, 
              early_stopping_rounds=params['od_wait'], cat_features=['fuel_type', 'model', 'car_type'])

    chusen_y_pred = chusen_model.predict(X_test)
        
    chusen_accuracy = accuracy_score(y_test, chusen_y_pred)
#     avg_prec = average_precision_score(y_test, y_pred)
    
    return chusen_accuracy

In [72]:
%%time 
chusen_study = optuna.create_study(pruner=HyperbandPruner(),
                            directions=['maximize'],
                           sampler=optuna.samplers.TPESampler(seed=42)) #pruner=MedianPruner(), sampler=optuna.samplers.TPESampler(seed=42)
chusen_study.optimize(chusen_objective,
               n_jobs=-1,
               n_trials=10, # 25 показывает результат лучше
               show_progress_bar=True
              )

chusen_best_trial = chusen_study.best_trials[0]
chusen_accuracy_best = chusen_best_trial.values[0]
# avg_prec_best = best_trial.values[1]

print(f"Лучшее значение  accuracy: {chusen_accuracy_best:.4f}")
# print(f"Лучшее значение Average Precision: {avg_prec_best:.4f}")

[I 2023-11-13 22:08:55,012] A new study created in memory with name: no-name-cea42019-ee54-4448-a462-5eaa660d7fd0


  0%|          | 0/10 [00:00<?, ?it/s]

[I 2023-11-13 22:18:12,811] Trial 3 finished with value: 0.9373219373219374 and parameters: {'iterations': 2984, 'boosting_type': 'Plain', 'depth': 9, 'learning_rate': 0.05547715176814741, 'l2_leaf_reg': 2.3587344545807425, 'border_count': 230, 'bagging_temperature': 6.572470363532972}. Best is trial 3 with value: 0.9373219373219374.
[I 2023-11-13 22:19:26,999] Trial 4 finished with value: 0.9401709401709402 and parameters: {'iterations': 1557, 'boosting_type': 'Plain', 'depth': 3, 'learning_rate': 0.07168824625377439, 'l2_leaf_reg': 7.265562996299715, 'border_count': 278, 'bagging_temperature': 7.268986142150059}. Best is trial 4 with value: 0.9401709401709402.
[I 2023-11-13 22:19:32,783] Trial 2 finished with value: 0.9430199430199431 and parameters: {'iterations': 1254, 'boosting_type': 'Plain', 'depth': 10, 'learning_rate': 0.0648553306965855, 'l2_leaf_reg': 13.725781312656272, 'border_count': 42, 'bagging_temperature': 4.33841062283102}. Best is trial 2 with value: 0.9430199430199

In [75]:
%%time
# Возьмите параметры из лучшего trial на основе первой метрики (ROC AUC в данном случае)
chusen_best_params = chusen_study.best_trials[0].params

chusen_CatBoost_model = CatBoostClassifier(**chusen_best_params,
                                    eval_metric='MultiClass',
#                                     auto_class_weights='Balanced',
                                    logging_level='Silent',
                                    use_best_model=True) #  task_type='GPU' auto_class_weights='Balanced' , 

chusen_CatBoost_model.fit(X_train, y_train,
                   eval_set=[(X_val, y_val)],
                   early_stopping_rounds=200,
                   verbose=100, cat_features=['fuel_type', 'model', 'car_type'])

chusen_y_pred = chusen_CatBoost_model.predict(X_test)
chusen_best_accuracy = accuracy_score(y_test, chusen_y_pred)
print(f"Лучшее значение  accuracy: {chusen_best_accuracy:.4f}")

Лучшее значение  accuracy: 0.9288
CPU times: user 1min 15s, sys: 4.41 s, total: 1min 19s
Wall time: 22.9 s


In [190]:
feature_importances = chusen_CatBoost_model.get_feature_importance(prettified=True)
feature_importances

,Feature Id,Importances
0,speed_max,38.245165
1,mean_rating,32.014198
2,rating_min,10.272324
3,speed_avg,5.365358
4,deviation_normal_mean,3.708277
5,deviation_normal_min,2.951524
6,user_uniq,2.919049
7,deviation_normal_max,2.885607
8,riders,1.638497


In [1]:
feature_importances

NameError: name 'feature_importances' is not defined